In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision 
from torch.optim import lr_scheduler
from torch.utils.data import  DataLoader,datasets,Subset
from torchvision import datasets, models, transforms
from PIL import Image
from torchvision import transforms
from PIL import Image
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt 
import torchvision.models as models
import copy

from tqdm import tqdm,trange
import os 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



from sklearn.linear_model import RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, f1_score


In [2]:
process_images = False
generate_datasets = False
generate_datasets_test = False

In [3]:
model = models.inception_v3(pretrained = True)
model.eval()

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stri

In [4]:
# sample execution (requires torchvision)
if process_images == True:

    for filename in tqdm(os.listdir("./task4_be9ai3nsdj/food/food/"), desc="Processing images..."):
    #     print(filename)
        if filename.endswith(".jpg"):

            input_image = Image.open("./task4_be9ai3nsdj/food/food/{}".format(filename))
            preprocess = transforms.Compose([
            transforms.Resize(299),
            transforms.CenterCrop(299),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
            input_tensor = preprocess(input_image)
            input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

    # move the input and model to GPU for speed if available
            if torch.cuda.is_available():
                input_batch = input_batch.to('cuda')
                model.to('cuda')
            output = model(input_batch)
            # Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
    #         print(output[0])
            # The output has unnormalized scores. To get probabilities, you can run a softmax on it.
            output_np = output.cpu().detach().numpy()
            np.save(
            "inception_model_processed/" + filename[:-4] + ".npy", output_np
            )

else:
    pass



In [5]:

indices = np.loadtxt("./task4_be9ai3nsdj/train_triplets.txt", dtype=str)
images_1 = [None] * indices.shape[0]
row = indices[0, :]
images_1[0] = np.load(f"inception_model_processed/{row[0]}.npy")
print(images_1[0])

[[-4.18649971e-01  7.19060659e-01 -4.18427199e-01 -7.97366142e-01
  -2.03729987e-01  1.45820111e-01 -9.56770182e-01  7.24522591e-01
   4.77914095e-01 -1.48355067e-01  6.10855296e-02 -9.45982337e-03
   7.29270577e-01 -3.94532382e-01  7.17636049e-01  3.16284448e-01
  -6.27331063e-02  1.09030414e+00  6.15721464e-01 -1.05012655e-02
  -1.99019998e-01 -7.58820832e-01 -4.02504712e-01 -7.06120253e-01
   8.56723636e-02 -1.61736384e-01 -8.49104047e-01 -9.63669419e-02
  -5.58910310e-01 -5.82792163e-02 -2.44810253e-01  6.39855087e-01
   5.83487272e-01  5.18645883e-01 -1.59742534e-01 -1.33142328e+00
   4.72966611e-01 -4.29704309e-01  4.82450187e-01 -1.50211048e+00
   4.86000717e-01 -4.20308143e-01 -4.08389717e-01  3.37118149e-01
  -1.14665031e-01 -4.32252198e-01 -2.57094651e-01 -1.20360970e-01
  -4.17623401e-01 -1.48686969e+00 -5.07346690e-01  8.69178772e-01
  -9.45716381e-01 -8.80110741e-01  1.76804245e-01  2.25659788e-01
   4.56566215e-02 -1.40401334e-01 -9.89916980e-01 -7.17310309e-01
  -1.19524

In [6]:
if generate_datasets == True:

    indices = np.loadtxt("./task4_be9ai3nsdj/train_triplets.txt", dtype=str)

    images_1 = [None] * indices.shape[0]
    images_2 = [None] * indices.shape[0]
    images_3 = [None] * indices.shape[0]

    for i in tqdm(range(indices.shape[0])):
        row = indices[i, :]
        images_1[i] = np.load(f"inception_model_processed/{row[0]}.npy")
        images_2[i] = np.load(f"inception_model_processed/{row[1]}.npy")
        images_3[i] = np.load(f"inception_model_processed/{row[2]}.npy")

    A = np.vstack(images_1)
    B = np.vstack(images_2)
    C = np.vstack(images_3)

    X1 = np.vstack((A, A, B))
    X2 = np.vstack((B, C, A))
    X3 = np.vstack((C, B, C))

    np.save("X1_InceptionV3.npy", X1)
    np.save("X2_InceptionV3.npy", X2)
    np.save("X3_InceptionV3.npy", X3)

X1 = np.load("X1_InceptionV3.npy").astype(np.float32)
X2 = np.load("X2_InceptionV3.npy").astype(np.float32)
X3 = np.load("X3_InceptionV3.npy").astype(np.float32)

dsize = X1.shape[0]
y_shape = (int(dsize / 3), 1)

Y = np.vstack((np.ones(y_shape), np.zeros(y_shape), np.ones(y_shape))).astype(np.float32)

In [7]:
if generate_datasets_test == True:

    indices = np.loadtxt("./task4_be9ai3nsdj/test_triplets.txt", dtype=str)

    images_1 = [None] * indices.shape[0]
    images_2 = [None] * indices.shape[0]
    images_3 = [None] * indices.shape[0]

    for i in tqdm(range(indices.shape[0])):
        row = indices[i, :]
        images_1[i] = np.load(f"inception_model_processed/{row[0]}.npy")
        images_2[i] = np.load(f"inception_model_processed/{row[1]}.npy")
        images_3[i] = np.load(f"inception_model_processed/{row[2]}.npy")

    X1_test = np.vstack(images_1)
    X2_test = np.vstack(images_2)
    X3_test = np.vstack(images_3)

    np.save("X1_test_InceptionV3.npy", X1_test)
    np.save("X2_test_InceptionV3.npy", X2_test)
    np.save("X3_test_InceptionV3.npy", X3_test)

X1_test = np.load("X1_test_InceptionV3.npy").astype(np.float32)
X2_test = np.load("X2_test_InceptionV3.npy").astype(np.float32)
X3_test = np.load("X3_test_InceptionV3.npy").astype(np.float32)

In [8]:
if X1 is not None:
    # If to ensure this cell isn't run twice.
    (
        X1_train,
        X1_validate,
        X2_train,
        X2_validate,
        X3_train,
        X3_validate,
        Y_train,
        Y_validate,
    ) = train_test_split(X1, X2, X3, Y, test_size=0.10, random_state=0, shuffle=True)

    # And freeing some memory
    del X1
    X1 = None
    del X2
    X2 = None
    del X3
    X3 = None


In [9]:
if not "scaler" in locals():
    scaler = StandardScaler()

    X1_train, X2_train, X3_train = np.vsplit(
        scaler.fit_transform(np.vstack((X1_train, X2_train, X3_train))), 3
    )

    # Transform the training data
    X1_validate = scaler.transform(X1_validate)
    X2_validate = scaler.transform(X2_validate)
    X3_validate = scaler.transform(X3_validate)

    # Transform the testing data
    X1_test = scaler.transform(X1_test)
    X2_test = scaler.transform(X2_test)
    X3_test = scaler.transform(X3_test)




In [10]:
device = torch.device("cuda")

In [11]:
class Densenet(torch.nn.Module):

    def __init__(self, input_layer,out_features,seed = 10):
        self.PW = 500
        self.DW = 200
        super().__init__()
        self.parelleblock= nn.Sequential(nn.Sequential(nn.Linear(in_dim, self.PW),nn.ReLU()),
                           nn.Sequential(nn.Linear(self.PW, self.PW),nn.ReLU()),
                           nn.Sequential(nn.Linear(self.PW, self.PW),nn.ReLU()))
        
        
        self.DB=nn.Sequential(nn.Sequential(nn.Linear(self.PW*3, self.DW ),nn.ReLU()),
                   nn.Sequential(nn.Linear(self.DW , self.DW ),nn.ReLU()),
                   nn.Sequential(nn.Linear(self.DW , 1 ),nn.Sigmoid())
                              )
        # self.net.to(device)
        
    def forward(self, im1,im2,im3):
        u1= self.parelleblock(im1)
        u2 = self.parelleblock(im2)
        u3 = self.parelleblock(im3)
#         print(u1.shape,u2.shape,u3.shape)
        cat = torch.cat((u1, u2,u3), dim=1) #keep track of dims
#         print(cat.shape)
        out = self.DB(cat).to(device)
        return out
    def predict(self, im1,im2,im3):
         return (self.forward(im1,im2,im3))
#         return nn.functional.softmax(self.forward(im1,im2,im3),dim=0)
# def forward(self, image, data):
#         x1 = self.cnn(image)
#         x2 = data
        
#         x = torch.cat((x1, x2), dim=1)
#         x = F.relu(self.fc1(x))
#         x = self.fc2(x)
#         return x

In [12]:
in_dim = X1_train.shape[1]
out_dim = Y_train.shape[1]
net_instance = Densenet(in_dim,out_dim)

In [13]:
out_dim

1

In [14]:
X1_train_tensor=torch.tensor(X1_train).to(device)
X2_train_tensor=torch.tensor(X2_train).to(device)
X3_train_tensor=torch.tensor(X3_train).to(device)
Y_train_tensor = torch.tensor(Y_train).to(device)
train_dataset = torch.utils.data.TensorDataset(X1_train_tensor,X2_train_tensor,X3_train_tensor,
                                            Y_train_tensor)
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=500)


In [15]:
net_instance.to(device)
for batch_x1,batch_x2,batch_x3,batch_y in train_dataloader:
    net_instance.forward(batch_x1,batch_x2,batch_x3)

In [35]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_function = nn.BCELoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [30]:
X1_validate_tensor=torch.tensor(X1_validate).to(device)
X2_validate_tensor=torch.tensor(X2_validate).to(device)
X3_validate_tensor=torch.tensor(X3_validate).to(device)
Y_validate_tensor = torch.tensor(Y_validate).to(device)
validate_dataset = torch.utils.data.TensorDataset(X1_validate_tensor,X2_validate_tensor,X3_validate_tensor,
                                            Y_validate_tensor)
validate_dataloader = torch.utils.data.DataLoader(validate_dataset,batch_size=500)

In [31]:
Y_validate_tensor

tensor([[0.],
        [1.],
        [1.],
        ...,
        [0.],
        [0.],
        [1.]], device='cuda:0')

In [32]:
X1_test_tensor=torch.tensor(X1_test).to(device)
X2_test_tensor=torch.tensor(X2_test).to(device)
X3_test_tensor=torch.tensor(X3_test).to(device)
test_dataset = torch.utils.data.TensorDataset(X1_test_tensor,X2_test_tensor,X3_test_tensor)
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=500)

In [33]:
# Run the training loop for defined number of epochs
def train_network(
    model,
    optimizer,
    scheduler,
    training_data_loader,
    testing_data_loader,
    training_loss_fn,
    testing_loss_fn,
    num_epochs,
):

    print(f"Batch size: {training_data_loader.batch_size}")

    pbar = trange(num_epochs)

    training_losses = []
    testing_losses = []

    for i in pbar:

        training_losses_batch = []
        for batch_x1,batch_x2,batch_x3,batch_y in training_data_loader:
            model.zero_grad()
            y_pred = model(batch_x1,batch_x2,batch_x3)
            loss = training_loss_fn(y_pred, batch_y)
            training_losses_batch.append(loss)
            pbar.set_description(f"loss {loss}")
            loss.backward()
#             criterion  = torch.mean(y_pred)
#             print(criterion)
            optimizer.step()
            scheduler.step()
        training_losses.append(
            torch.mean(torch.stack(training_losses_batch)).detach()
        )

        testing_losses_batch = []
        for test_x1, test_x2,test_x3,test_y in testing_data_loader:
            y_pred_test = model( test_x1, test_x2,test_x3)
            testing_losses_batch.append(testing_loss_fn(y_pred_test, test_y))

        testing_losses.append(
            torch.mean(torch.stack(testing_losses_batch)).detach()
        )

    return (
        torch.stack(training_losses).detach().cpu().numpy(),
        torch.stack(testing_losses).detach().cpu().numpy(),
    )


In [36]:

training_losses, testing_losses = train_network(net_instance,optimizer,scheduler,train_dataloader,
                                                validate_dataloader,loss_function,
                                                loss_function,50)

loss 0.6843823194503784:   0%|                                                                  | 0/50 [00:00<?, ?it/s]

Batch size: 500


loss 0.6829522848129272:   0%|                                                                  | 0/50 [00:02<?, ?it/s]


KeyboardInterrupt: 

tensor(0.5684, device='cuda:0')

In [ ]:
saved_network = torch.save(net_instance.state_dict(),"fitted net")

In [ ]:
prediction = []
for batch_x1,batch_x2,batch_x3 in test_dataloader:
    prediction_temp = net_instance.predict(batch_x1,batch_x2,batch_x3)
    prediction_temp = prediction_temp.cpu().detach().numpy()
    prediction = np.append(prediction,prediction_temp)

prediction_rounded = np.round(prediction)

In [ ]:
loaded_model = models.Densenet(
    input_size=n_components,
    num_layers=n_layers,
    width=width,
    output_dim=n_output,
)

In [51]:
pd.DataFrame(prediction.T).to_csv('prediction.csv', index=False, header=None)

In [ ]:
testing_losses

In [31]:
# prediction.max()
torch.cuda.empty_cache()